In [2]:
import simpy 
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd

In [46]:
"""
Bank renege example

Covers:

- Resources: Resource
- Condition events

Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

"""
import random
import simpy

waiting_times = []
service_times = []

def source(env, number, lambd, mu, counter, B_distribution, shortest_job_first=False):
    """Source generates customers randomly"""
    t0 = np.random.exponential(1/lambd) # Arrival time of first customer
    yield env.timeout(t0)

    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, mu, B_distribution, shortest_job_first)
        env.process(c)

        # A distribution
        t = np.random.exponential(1/lambd)
        yield env.timeout(t)
   
def customer(env, name, counter, mu, B_distribution, shortest_job_first=False):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    # print('%7.4f %s: Here I am' % (arrive, name))

    priority = service_time if shortest_job_first else 1

    with counter.request(priority=priority) as req: 
        yield req
        wait = env.now - arrive
        waiting_times.append(wait)
        # print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
        service_time = np.random.exponential(1/mu)
        service_times.append(service_time)
        yield env.timeout(service_time)
        # print('%7.4f %s: Finished' % (env.now, name))


In [42]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 10000 # Total number of customers
LAMBD = 1/12
MU = 1/11
random.seed(RANDOM_SEED)

def mmc_sim(new_customers, lambd, mu, capacity, B_distribution, shortest_job_first=False):
    global waiting_times
    global service_times
    waiting_times = []
    service_times = []
    
    env = simpy.Environment()

    # Start processes and run
    counter = simpy.PriorityResource(env, capacity=capacity)
    env.process(source(env, new_customers, lambd, mu, counter, B_distribution, shortest_job_first))
    env.run()
    
    return waiting_times, service_times

22.4349 Customer00: Here I am
22.4349 Customer00: Waited  0.000
25.2589 Customer00: Finished
28.6390 Customer01: Here I am
28.6390 Customer01: Waited  0.000
42.5750 Customer01: Finished
50.8988 Customer02: Here I am
50.8988 Customer02: Waited  0.000
51.0139 Customer02: Finished
59.7334 Customer03: Here I am
59.7334 Customer03: Waited  0.000
76.6625 Customer03: Finished
82.0768 Customer04: Here I am
82.0768 Customer04: Waited  0.000
91.6571 Customer04: Finished
[0.0, 0.0, 0.0, 0.0, 0.0]


In [312]:
def queueing_simulations(new_customers, lambd, mu, capacity, n_samples):
    
    dict_wt = {}
    dict_st = {}
    for i in range(n_samples):
        global waiting_times
        global service_times
        waiting_times = []
        service_times = []
        waiting_times, service_times = mmc_sim(new_customers, lambd, mu, capacity)
        dict_wt[i] = waiting_times
        dict_st[i] = service_times


    return dict_wt, dict_st

result = queueing_simulations(5, 1/12, 1/11, 1, 3)
print(result[0])

 5.0746 Customer00: Here I am
 5.0746 Customer00: Waited  0.000
19.0594 Customer01: Here I am
24.5910 Customer02: Here I am
27.0507 Customer03: Here I am
37.6224 Customer00: Finished
37.6224 Customer01: Waited 18.563
80.4114 Customer01: Finished
80.4114 Customer02: Waited 55.820
81.9333 Customer04: Here I am
83.2974 Customer02: Finished
83.2974 Customer03: Waited 56.247
86.3473 Customer03: Finished
86.3473 Customer04: Waited  4.414
87.0532 Customer04: Finished
 2.5915 Customer00: Here I am
 2.5915 Customer00: Waited  0.000
 4.1706 Customer00: Finished
11.1291 Customer01: Here I am
11.1291 Customer01: Waited  0.000
11.2478 Customer01: Finished
22.6661 Customer02: Here I am
22.6661 Customer02: Waited  0.000
29.4887 Customer02: Finished
47.2792 Customer03: Here I am
47.2792 Customer03: Waited  0.000
58.2707 Customer04: Here I am
61.1593 Customer03: Finished
61.1593 Customer04: Waited  2.889
65.5542 Customer04: Finished
15.7973 Customer00: Here I am
15.7973 Customer00: Waited  0.000
16.227

In [47]:
# Start out with mu = 0.1, rho = 0.9

# Variation of simulation function where you input rho and the function calculates a lambda 
def queueing_simulations2(new_customers, rho, mu, capacity, n_samples):
    
    lambd = rho * capacity * mu 
    dict_wt = {}
    dict_st = {}
    for i in range(n_samples):
        print(i)
        global waiting_times
        global service_times
        waiting_times = []
        service_times = []
        waiting_times, service_times = mmc_sim(new_customers, lambd, mu, capacity)
        dict_wt[i] = waiting_times
        dict_st[i] = service_times


    return dict_wt, dict_st



In [313]:
result_n1 = queueing_simulations2(1000, 0.9, 0.1, 1, 50)
result_n2 = queueing_simulations2(1000, 0.9, 0.1, 2, 50)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [53]:
def compute_meanwt(result_dict):
    l_mean_wt = []
    for key in result_dict.keys(): 
        l_mean_wt.append(np.mean(result_dict[key]))
    mean_wt = np.mean(l_mean_wt)
    return mean_wt

def compute_varwt(result_dict):
    l_mean_wt = []
    for key in result_dict.keys(): 
        l_mean_wt.append(np.mean(result_dict[key]))
    var_wt = np.var(l_mean_wt)
    return var_wt

print(compute_meanwt(result_n1[0]))
print(compute_varwt(result_n1[0]))
print(compute_meanwt(result_n2[0]))
print(compute_varwt(result_n2[0]))


77.16910687277583
1242.1423163757036
36.4692837468269
345.61634992264607


In [ ]:
p_0_inverse = 0
rho = LAMBD/(c*MU)

for n in range(c):
    p_0_inverse += (c*rho)**n/math.factorial(n) + ((c*rho)**c/math.factorial(c))/(1-rho)

p_0 = 1/p_0_inverse
p_c = (c*rho)**c/math.factorial(c) * p_0
delay_probability = p_c/(1-rho)
E_W = delay_probability * (1/(1-rho)) * 1/(c*MU)

print(f'Theoretical mean waiting time = {E_W}')
print(f'Empirical mean waiting time = {np.mean(waiting_times)}')